In [1]:
from tensorflow.keras.layers import SimpleRNN, LSTM, GRU, Bidirectional, Dense, Embedding


In [2]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
import numpy as np

In [3]:
# Getting reviews with words that come under 5000 most occurring words in the entire corpus of textual review data
vocab_size = 5000

In [4]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size)
print(x_train[0])

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 2, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 2, 19, 178, 32]


In [5]:
# Getting all the words from word_index dictionary
word_idx = imdb.get_word_index()

1641221/1641221 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [6]:
#Originally the index number of a value and not a key, hence converting the index as key and the words as values
word_idx = {i: word for word, i in word_idx.items()}

# word and i ==> word=key, i=value

In [7]:
# again printing the review
print([word_idx[i] for i in x_train[0]])

['the', 'as', 'you', 'with', 'out', 'themselves', 'powerful', 'lets', 'loves', 'their', 'becomes', 'reaching', 'had', 'journalist', 'of', 'lot', 'from', 'anyone', 'to', 'have', 'after', 'out', 'atmosphere', 'never', 'more', 'room', 'and', 'it', 'so', 'heart', 'shows', 'to', 'years', 'of', 'every', 'never', 'going', 'and', 'help', 'moments', 'or', 'of', 'every', 'chest', 'visual', 'movie', 'except', 'her', 'was', 'several', 'of', 'enough', 'more', 'with', 'is', 'now', 'current', 'film', 'as', 'you', 'of', 'mine', 'potentially', 'unfortunately', 'of', 'you', 'than', 'him', 'that', 'with', 'out', 'themselves', 'her', 'get', 'for', 'was', 'camp', 'of', 'you', 'movie', 'sometimes', 'movie', 'that', 'with', 'scary', 'but', 'and', 'to', 'story', 'wonderful', 'that', 'in', 'seeing', 'in', 'character', 'to', 'of', '70s', 'and', 'with', 'heart', 'had', 'shadows', 'they', 'of', 'here', 'that', 'with', 'her', 'serious', 'to', 'have', 'does', 'when', 'from', 'why', 'what', 'have', 'critics', 'they'

In [8]:
# Get the minimum and the maximum length of reviews
print("Max length of a review:: ", len(max((x_train+x_test), key=len)))
print("Min length of a review:: ", len(min((x_train+x_test), key=len)))

Max length of a review::  2697
Min length of a review::  70


#key specifies how the elements should be compared. Here, len ensures that the comparison is based on the length of each review, not the content of the review.

In [9]:
from tensorflow.keras.preprocessing import sequence

In [10]:
# Keeping a fixed length of all reviews to max 400 words
max_words = 400

x_train = sequence.pad_sequences(x_train, maxlen=max_words)
x_test = sequence.pad_sequences(x_test, maxlen=max_words)

x_valid, y_valid = x_train[:64], y_train[:64]
x_train_, y_train_ = x_train[64:], y_train[64:]

sequence.pad_sequences: Ensures that all sequences in x_train and x_test have the same length (max_words in this case).
Reviews longer than max_words will be truncated (usually from the beginning or end, depending on settings).
Reviews shorter than max_words will be padded with zeros (or another specified value) to the left or right (default is to pad on the left).

# why padding imp
Machine learning models, particularly neural networks, require a fixed input size for efficient matrix operations.
During training, models process data in batches for computational efficiency.
If sequences within a batch have different lengths, it becomes impossible to create a single tensor to feed the model.

# x_valid, y_valid = x_train[:64], y_train[:64]
The first 64 samples of the training data (x_train) and their corresponding labels (y_train) are set aside as a validation set (x_valid, y_valid).
Validation data is used to evaluate the model during training to check its performance on unseen data.

# x_train_, y_train_ = x_train[64:], y_train[64:]
The remaining training data (excluding the first 64 samples) is used for actual training (x_train_, y_train_).

In [12]:
# fixing every word's embedding size to be 32
embd_len = 32

# Creating a RNN model
RNN_model = Sequential(name="Simple_RNN")  #initialize

RNN_model.add(Embedding(vocab_size,
                        embd_len,
                        input_length=max_words))

# In case of a stacked(more than one layer of RNN)
# use return_sequences=True
RNN_model.add(SimpleRNN(128,
                        activation='tanh',
                        return_sequences=False))

RNN_model.add(Dense(1, activation='sigmoid'))

# printing model summary
print(RNN_model.summary())

# Compiling model
RNN_model.compile(
    loss="binary_crossentropy",
    optimizer='adam',
    metrics=['accuracy']
)

# Training the model
history = RNN_model.fit(x_train_, y_train_,
                        batch_size=64,
                        epochs=5,
                        verbose=1,
                        validation_data=(x_valid, y_valid))


C:\Users\cchin\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "Simple_RNN"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn (SimpleRNN)               │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/5
390/390 ━━━━━━━━━━━━━━━━━━━━ 49s 115ms/step - accuracy: 0.5286 - loss: 0.6882 - val_accuracy: 0.6250 - val_loss: 0.6537
Epoch 2/5
390/390 ━━━━━━━━━━━━━━━━━━━━ 44s 112ms/step - accuracy: 0.6834 - loss: 0.5876 - val_accuracy: 0.5938 - val_loss: 0.6444
Epoch 3/5
390/390 ━━━━━━━━━━━━━━━━━━━━ 44s 114ms/step - accuracy: 0.6902 - loss: 0.5885 - val_accuracy: 0.6406 - val_loss: 0.6069
Epoch 4/5
390/390 ━━━━━━━━━━━━━━━━━━━━ 45s 114ms/step - accuracy: 0.7884 - loss: 0.4565 - val_accuracy: 0.8125 - val_loss: 0.4434
Epoch 5/5
390/390 ━━━━━━━━━━━━━━━━━━━━ 44s 112ms/step - accuracy: 0.8126 - loss: 0.4235 - val_accuracy: 0.5938 - val_loss: 0.6502


In [13]:
# Printing model score on test data
print()
print("Simple_RNN Score---> ", RNN_model.evaluate(x_test, y_test, verbose=0))


Simple_RNN Score--->  [0.6111391186714172, 0.6661199927330017]


# accuracy===>66.61%